In [1]:
#opendatasets: fetch from kaggle
#datasets: convert the database so that transformers library can understand easily
#used to get data from huggingface as well
#transformer:
#bitsandbytes: quantization
#accelerate: bitsandbytes depends on accelerate
#perft: parameter efficient fine tuning, used for LoRA

!pip install opendatasets datasets transformers bitsandbytes accelerate peft --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: halukcelebi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail


100%|██████████| 503M/503M [00:24<00:00, 21.2MB/s]


In [3]:
import os
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM,AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig,Trainer
from peft import LoraConfig, get_peft_model
import re
import pandas as pd

In [4]:
data_df = pd.read_csv("/content/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")[["article", "highlights"]]
data_df = data_df.sample(10000)
data_df.head()

,article,highlights
76320,(CNN) -- Northern Ireland's Rory McIlroy storm...,Rory McIlroy shoots seven-under 63 in opening ...
81838,(CNN) -- A suicide car bomber killed four peop...,27 people also injured when a suicide bomber d...
266679,"LONDON, England (CNN) -- You're sitting in an ...",Security experts warn Wi-Fi users to be more v...
84073,After Natalie Morris broke her back in a horri...,"Natalie Morris, 26, fell down the stairs going..."
246123,(CNN) -- Golf fans grew accustomed to the sigh...,Javier Ballesteros wins the amateur Madrid Ope...


In [5]:
def filter_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+',' ',text)
  return text

In [6]:
filter_text("Who Are you *?!")

'who are you '

In [7]:
data_df["article"] = data_df["article"].apply(filter_text)
data_df["highlights"] = data_df["highlights"].apply(filter_text)
data_df.head()

,article,highlights
76320,cnn northern ireland s rory mcilroy stormed t...,rory mcilroy shoots seven under 63 in opening ...
81838,cnn a suicide car bomber killed four people a...,27 people also injured when a suicide bomber d...
266679,london england cnn you re sitting in an airpor...,security experts warn wi fi users to be more v...
84073,after natalie morris broke her back in a horri...,natalie morris 26 fell down the stairs going t...
246123,cnn golf fans grew accustomed to the sight of...,javier ballesteros wins the amateur madrid ope...


In [8]:
data_df["final_statement"] = ""
data_df.head()

,article,highlights,final_statement
76320,cnn northern ireland s rory mcilroy stormed t...,rory mcilroy shoots seven under 63 in opening ...,
81838,cnn a suicide car bomber killed four people a...,27 people also injured when a suicide bomber d...,
266679,london england cnn you re sitting in an airpor...,security experts warn wi fi users to be more v...,
84073,after natalie morris broke her back in a horri...,natalie morris 26 fell down the stairs going t...,
246123,cnn golf fans grew accustomed to the sight of...,javier ballesteros wins the amateur madrid ope...,


In [9]:
data_df['final_statement'] = data_df.apply(
    lambda row: "Summarize the following article: \n\n" + str(row['article']) + "\nSummary:" + str(row['highlights']),
    axis=1
)

In [10]:
#for indx, row in data_df.iterrows():
#  row["final_statement"] = "Summarize the following article: \n\n" +str(row["article"]) + "\nSummary:" +str(row["highlights"])
#  data_df["final_statement"].iloc[i] = "Summarize the following article: \n\n" +str(row["article"]) + "\nSummary:" +str(row["highlights"])

In [11]:
data_df.sample()["final_statement"].iloc[0]

'Summarize the following article: \n\nclick here to see our brilliant match zone including a pitch map of charlie austin s all action performance manuel pellegrini will be thankful for two things on sunday morning one it is the end of a difficult week and two sergio aguero plays for manchester city the argentine yet again dug his team out of a huge hole with a breathtaking double as city twice came from behind to snatch a draw at queens park rangers video scroll down for sportsmail s premier league big match stats qpr vs manchester city charlie austin who had already had two goals disallowed scores to put qpr ahead at home to the premier league champions sergio aguero follows up his brilliant control to fire past the despairing lunge of steven caulker and level the scores at loftus road bobby zamora celebrates after martin demichelis had headed into his own net to put qpr back ahead with 15 minutes to play aguero stays calm after rounding rob green to level for city again as the champi

In [12]:
data_df = data_df[["final_statement"]]
data_df.head()

,final_statement
76320,Summarize the following article: \n\n cnn nort...
81838,Summarize the following article: \n\n cnn a su...
266679,Summarize the following article: \n\nlondon en...
84073,Summarize the following article: \n\nafter nat...
246123,Summarize the following article: \n\n cnn golf...


In [13]:
tokenizer= AutoTokenizer.from_pretrained("bigscience/bloom-1b1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [14]:
tokenizer.pad_token #if not have it, must set it
#tokenizer.pad_token = tokenizer.eos_token

'<pad>'

In [15]:
def tokenize_function(example):
  example['input_ids'] = tokenizer(example["final_statement"], padding = "max_length",max_length = 512, truncation = True, return_tensors = 'pt').input_ids
  example['labels'] = tokenizer(example["final_statement"], padding = "max_length",max_length = 512, truncation = True, return_tensors = 'pt').input_ids
  return example

In [16]:
train_data = Dataset.from_pandas(data_df)
train_tokenized_dataset = train_data.map(tokenize_function, batched = True, remove_columns = train_data.column_names)
train_tokenized_dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10000
})

In [17]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4"
)
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b1", quantization_config = quant_config)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

In [18]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 8,
    bias = "none",
    task_type = "CAUSAL_LM"
)
peft_model = get_peft_model(model, peft_params)

In [19]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1536)
        (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=4608, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4608, bias=False)
                )
                

In [20]:
print("PEFT MODEL", peft_model.print_trainable_parameters() )

trainable params: 1,179,648 || all params: 1,066,493,952 || trainable%: 0.1106
PEFT MODEL None


In [21]:
training_args = TrainingArguments(
    output_dir = "./model_checkpoints",
    save_total_limit = 1,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 5
)

trainer = Trainer(
    model = peft_model,
    args = training_args,
    train_dataset = train_tokenized_dataset
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
500,3.755100
1000,3.745900
1500,3.695000
2000,3.703100
2500,3.661600
3000,3.656700
3500,3.620800
4000,3.634900
4500,3.676200
5000,3.676400


TrainOutput(global_step=12500, training_loss=3.61436140625, metrics={'train_runtime': 26453.296, 'train_samples_per_second': 1.89, 'train_steps_per_second': 0.473, 'total_flos': 1.046234529792e+17, 'train_loss': 3.61436140625, 'epoch': 5.0})

In [22]:
trainer.model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.json')

In [23]:
tokenizer= AutoTokenizer.from_pretrained("final_model")
trained_model = AutoModelForCausalLM.from_pretrained("final_model").to("cuda")

In [28]:
article = """Summarize the following article:\n\n
Doug and I decided to end our marriage for a variety of reasons, many years ago," Kerstin Emhoff told the Washington Post in a statement Saturday. "He is a great father to our kids, continues to be a great friend to me and I am really proud of the warm and supportive blended family Doug, Kamala, and I have built together."

Emhoff admitted to having an affair with a nanny shortly after the Daily Mail published a report last week that the second gentleman had an affair with his daughter's nanny and got her pregnant. The nanny's close friend told the outlet that she did not keep the baby, but did not elaborate further.

"During my first marriage, Kerstin and I went through some tough times on account of my actions. I took responsibility, and in the years since, we worked through things as a family and have come out stronger on the other side," Emhoff told CNN last week of the affair.
\nSummary:\n
"""

article = filter_text(article)
input_ids = tokenizer(article, padding = "max_length", max_length = 250, truncation = True, return_tensors = 'pt').input_ids.to('cuda')


In [29]:
outputs = trained_model.generate(input_ids, max_new_tokens = 30)

In [30]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'summarize the following article doug and i decided to end our marriage for a variety of reasons many years ago kerstin emhoff told the washington post in a statement saturday he is a great father to our kids continues to be a great friend to me and i am really proud of the warm and supportive blended family doug kamala and i have built together emhoff admitted to having an affair with a nanny shortly after the daily mail published a report last week that the second gentleman had an affair with his daughter s nanny and got her pregnant the nanny s close friend told the outlet that she did not keep the baby but did not elaborate further during my first marriage kerstin and i went through some tough times on account of my actions i took responsibility and in the years since we worked through things as a family and have come out stronger on the other side emhoff told cnn last week of the affair summary erstwhile nanny emhoff s close friend told the washington post that she did not keep th